In [ ]:
import argparse
from argparse import ArgumentParser
import glob
import cv2
import re
import os, glob, datetime
import numpy as np
import tensorflow as tf
import numpy as np
import time
import math
from keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract, Reshape
from keras.models import Model, load_model
from tensorflow.python.keras.utils import conv_utils
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
import keras.backend as K

import skimage
from skimage.metrics import structural_similarity as ssim
from skimage.io import imread, imsave


##--------------------------------------------------------------------------------------------------------
##--------------------------------------------------------------------------------------------------------


#########--------------           IMPORTANT NOTE       ----------------###############

# Please download the pretrained model from 
#   https://www.dropbox.com/s/6edzcf76lnfn7uj/DIVA_US_image_despeckling_model.hdf5?dl=0
#   and save file into 'Model' folder before testing

# Select the path of the directory properly to give access to the models, datasets and results
directory_path =  '___________________________________' #'Choose your directory




parser = argparse.ArgumentParser()
# choose if needed
parser.add_argument('--model', default='DIVA', type=str, help='choose a type of model')

parser.add_argument('--set_dir', default=os.path.join(directory_path ,'Data/'), type=str, help='directory of test dataset')
parser.add_argument('--Speckled_set_names', default=['test_Images'], type=list, help='name of the speckled test dataset')


#########--------------           IMPORTANT NOTE       ----------------###############

# read the clean data set if available
#parser.add_argument('--set_names_clean', default='More_test_Images_Original', type=str, help='name of the clean test dataset')

parser.add_argument('--model_dir', default=os.path.join(directory_path ,'Model'), type=str, help='directory of the model')
                                                     
parser.add_argument('--model_name', default='DIVA_US_image_despeckling_model.hdf5', type=str, help='the model name')
parser.add_argument('--result_dir', default=os.path.join(directory_path,'results'), type=str, help='directory of results')
parser.add_argument('--save_result', default=1, type=int, help='save the denoised image, 1 or 0')

parser.add_argument('-f', '--file', required=False)
args = parser.parse_args()



print(args.model_name)

## -------------------------------------------------------------------------------------------------------------

##------------------------------------------------------------------------------------------------------------


# Define useful functions

def to_tensor(img):
    if img.ndim == 2:
        return img[np.newaxis,...,np.newaxis]
    elif img.ndim == 3:
        return np.moveaxis(img,2,0)[...,np.newaxis]

def from_tensor(img):
    return np.squeeze(np.moveaxis(img[...,0],0,-1))

def log(*args,**kwargs):
     print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S:"),*args,**kwargs)

def save_result(result,path):
    path = path if path.find('.') != -1 else path+'.png'
    ext = os.path.splitext(path)[-1]
    if ext in ('.txt','.dlm'):
        np.savetxt(path,result,fmt='%2.4f')
    else:
        imsave(path,np.clip(result,0,1))


def show(x,title=None,cbar=False,figsize=None):
    import matplotlib.pyplot as plt
    plt.figure(figsize=figsize)
    plt.imshow(x,interpolation='nearest',cmap='gray')
    if title:
        plt.title(title)
    if cbar:
        plt.colorbar()
    plt.show()

def psnr(target, ref):
    # Assume target is RGB/BGR image
    target_data = target.astype(np.float32)
    ref_data = ref.astype(np.float32)
    
    diff = ref_data - target_data
    diff = diff.flatten('C')
    
    rmse = np.sqrt(np.mean(diff ** 2.))
    
    return 20 * np.log10(1. / rmse)


def snr(target, ref):
    # Assume target is RGB/BGR image
    target_data = target.astype(np.float32)
    ref_data = ref.astype(np.float32)
    
    diff = ref_data - target_data
    diff = diff.flatten('C')
    target_data = target_data.flatten('C')
    
    rmse_diff = np.sqrt(np.mean(diff ** 2.))
    rmse_target = np.sqrt(np.mean(target_data ** 2.))

    return 20 * np.log10(rmse_target / rmse_diff)

##----------------------------------------------------------------------------------------------------------------

##----------------------------------------------------------------------------------------------------------------


if __name__ == '__main__':    
    
    # Load trained model
    print('Model- ',args.model)
    model = load_model(os.path.join(args.model_dir, args.model_name),compile=False)
    log('load trained model')
    
    # Make directory for results
    if not os.path.exists(args.result_dir):
        os.mkdir(args.result_dir)

#-----------------------------------------------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------------------------------------


    for set_cur in args.Speckled_set_names:  
        
        if not os.path.exists(os.path.join(args.result_dir,set_cur)):
            os.mkdir(os.path.join(args.result_dir,set_cur))

        
        for im in os.listdir(os.path.join(args.set_dir,set_cur)): 
            if im.endswith(".jpg") or im.endswith(".bmp") or im.endswith(".png"):

                # speckled noise case
                noise_type = 'Speckled'

                # read the speckled image
                y = np.array(imread(os.path.join(args.set_dir,set_cur,im)), dtype=np.float32) / 255.0


                y = y.astype(np.float32)
                y_  = to_tensor(y)
                start_time = time.time()
                x_ = model.predict(y_)      # Despeckling process
                elapsed_time = time.time() - start_time
                print('%10s : %10s : %2.4f second'%(set_cur,im,elapsed_time))

                # Despeckled image
                x_=from_tensor(x_)


                if args.save_result:
                    name, ext = os.path.splitext(im)
                    show(np.hstack((y,x_))) # show the image
                    save_result(x_,path=os.path.join(args.result_dir,set_cur,name+ext)) # save the Despeckled image
                
#-----------------------------------------------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------------------------------------

